<a href="https://colab.research.google.com/github/OmarK920/Deep-Learning-Projects/blob/main/ImageRecognition_FaceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trainig SVC

In [ ]:
import io
import cv2
import zipfile
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from skimage.feature import hog

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
new_width, new_height = 350, 300

with zipfile.ZipFile('img.zip') as zf:
    filenames = zf.namelist()
    images = []
    for filename in filenames:
        with zf.open(filename) as f:
            img_bytes = io.BytesIO(f.read())
            image = Image.open(img_bytes)
            image = image.resize((new_width, new_height))
            img_array = np.array(image)
            image = Image.fromarray(img_array)
            images.append(image)

labels = ["huss", "huss", "huss", "huss", "nour", "nour", "nour", "nour", "omar", "omar", "omar", "omar"]
data = {"label": labels, "image": images}
df = pd.DataFrame(data)

images = []
labels = []

for index, row in df.iterrows():
    image = row['image']
    images.append(image)

X = np.empty((0, 315000))
for image in images:
    image_array = np.array(image).flatten()
    X = np.vstack([X, image_array])

for index, row in df.iterrows():
    label = row['label']
    labels.append(label)

y = np.array(labels)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm = SVC(kernel='sigmoid', random_state=42)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)
print("Confusion Matrix:\n", confusion_mat)


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

Confusion Matrix:
 [[1 0]
 [0 2]]


# Using HOG with SVC

In [ ]:
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Extract HOG features from images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        # Convert image to grayscale if necessary
        if len(image.shape) > 2:
            image = np.mean(image, axis=2)
        # Extract HOG features
        hog_feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(hog_feature)
    return np.array(hog_features)

# Prepare your data (replace with your own data)
labels = ["huss", "huss", "huss", "huss", "nour", "nour", "nour", "nour", "omar", "omar", "omar", "omar"]
data = {"label": labels, "image": images}
df = pd.DataFrame(data)

# Convert images to NumPy arrays
images = np.array([np.array(img) for img in df["image"]], dtype=object)

# Extract HOG features
X_hog = extract_hog_features(images)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, labels, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an SVM model
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Predict labels for test data
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print("Accuracy:", accuracy)


Accuracy: 1.0


# Training CNN

In [ ]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from skimage.feature import hog



# Define the CNN model architecture
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(new_height, new_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3 , activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

labels = ["huss", "huss", "huss", "huss", "nour", "nour", "nour", "nour", "omar", "omar", "omar", "omar"]
data = {"label": labels, "image": images}
df = pd.DataFrame(data)

images = []
labels = []

for index, row in df.iterrows():
    image = row['image']
    images.append(image)

for index, row in df.iterrows():
    label = row['label']
    labels.append(label)

# Convert the images and labels to NumPy arrays
X = np.stack(images)
y = np.array(labels)


# Normalize pixel values (assuming pixel range 0-255)
X = X / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to integers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)







Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 3s 3s/step - loss: 1.1427 - accuracy: 0.1111 - val_loss: 100.2899 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 37.4026 - accuracy: 0.4444 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 10.1390 - accuracy: 0.5556 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 7.5937 - accuracy: 0.5556 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 2.2326 - val_accuracy: 0.3333
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 0.5089 - accuracy: 0.8889 - val_loss: 19.7571 - val_accuracy: 0.3333
Epoch 8/10
1/1 [===

## CNN with HOG

In [ ]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.model_selection import train_test_split

# Function to extract HOG features
def extract_hog_features(images):
    hog_features = []
    for image in images:
        hog_feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1)
        hog_features.append(hog_feature)
    return np.array(hog_features)

# Define the CNN model architecture
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(54432,)))  # Adjust the input shape based on the HOG feature length
model.add(layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

labels = ["huss", "huss", "huss", "huss", "nour", "nour", "nour", "nour", "omar", "omar", "omar", "omar"]
data = {"label": labels, "image": images}
df = pd.DataFrame(data)

images = []
labels = []

for index, row in df.iterrows():
    image = row['image']
    images.append(image)

for index, row in df.iterrows():
    label = row['label']
    labels.append(label)

# Convert the images and labels to NumPy arrays
X = np.stack(images)
y = np.array(labels)

# Normalize pixel values (assuming pixel range 0-255)
X = X / 255.0

# Extract HOG features
hog_features = extract_hog_features(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, y, test_size=0.2, random_state=42)

# Convert labels to integers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 [==============================] - 1s 770ms/step - loss: 1.1532 - accuracy: 0.2222 - val_loss: 10.6158 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 0s 86ms/step - loss: 3.3954 - accuracy: 0.7778 - val_loss: 5.6495 - val_accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 0s 93ms/step - loss: 2.2122 - accuracy: 0.5556 - val_loss: 0.1222 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 88ms/step - loss: 0.0216 - accuracy: 1.0000 - val_loss: 0.4857 - val_accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 74ms/step - loss: 0.0452 - accuracy: 1.0000 - val_loss: 1.0699 - val_accuracy: 0.6667
Epoch 6/10
1/1 [==============================] - 0s 91ms/step - loss: 0.2115 - accuracy: 0.8889 - val_loss: 0.4896 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 78ms/step - loss: 0.0342 - accuracy: 1.0000 - val_loss: 0.0816 - val_accuracy: 1.0000
Epoch 8/10
1/1 [====================

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc = nn.Linear(16 * 87 * 75, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Convert labels to a 1D array
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['image'], y, test_size=0.2, random_state=42)

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Create custom datasets and data loaders
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Create an instance of the CNN model
model = SimpleCNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1} - Loss: {running_loss / len(train_loader)}")

# Evaluation
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)
